# Final project code

In [1]:
import json
import numpy as np
import re
import copy
import gensim
import nltk
import string 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
import gensim.downloader as api

In [45]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
with open('training_jsons/apple.json', 'r') as f:
    apple_data = json.load(f)
with open('training_jsons/google.json', 'r') as f:
    google_data = json.load(f)
with open('training_jsons/microsoft.json', 'r') as f:
    microsoft_data = json.load(f)

In [3]:
data = apple_data + google_data + microsoft_data

In [4]:
print(data[232]) #Remove \r\n(-: 

{'offer_status': 'No Offer', 'experience': 'Negative Experience', 'difficulty': 'Easy Interview', 'review': '10 minute speed run, very abrupt with unengaged interviewers. Asked behavioral questions and about previous projects. Describe a previous coding project, interests, Why Apple? \n\nDid not find this format engaging, thought I do like the efficiency of the structure. Interviewers did not seem to care or have enough time to get to know candidates', 'page': 120}


In [5]:
#Types of offers
offer = set()
experience = set()
difficulty = set()
for i in data:
    offer.add(i["offer_status"])
    experience.add(i["experience"])
    difficulty.add(i["difficulty"])
print(offer)
print(experience)
print(difficulty)

{'Declined Offer', 'Accepted Offer', 'No Offer'}
{'Negative Experience', 'Neutral Experience', 'Positive Experience'}
{'Easy Interview', 'Average Interview', 'Difficult Interview'}


In [6]:
y = []
X = []
#0 represents no offer
#1 represents declined offer or accepted offer
#Decide if we should predict offer or the experience?
#Preprocessing and creating y_matrix

#Future step: Tokenize to remove stop words to only include stems of words?



for i in range(len(data)):
    #Remove \w \s \n
    data[i]["review"] = data[i]["review"].strip().replace('\n', '').replace('\r', '').lower()
    data[i]["review"] = re.sub(r'[^\w\s\n\r]', '', data[i]["review"]).strip()
    #Remove links
    data[i]["review"] = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', data[i]["review"])
    if data[i]["offer_status"] == 'No Offer':
        data[i]["offer_status"] = 0
        y.append(0)
    else:
        data[i]["offer_status"] = 1
        y.append(1)
    del data[i]['offer_status']
print(y[1:10])
print(data[0])

[1, 1, 1, 1, 0, 1, 0, 1, 1]
{'experience': 'Positive Experience', 'difficulty': 'Average Interview', 'review': 'initial phone interview 15mins received an email 2 days after applying inviting me to schedule a time to call with an apple retail recruiter who was based in california applying for job in texas was told at the end of the phone call that i would be advancing to the next step invited to an optional get to know apple web event 30mins which went over some basics about working in apple retail and company culturegroup interview 45mins group interview with manager from the apple store i was applying to and 3 other interviewees 4 interviewees total took place online via their webex platform similar to zoom as of aug 2021 most interviewees were online 105 minutes before the interview was scheduled to begin and an icebreaker question was asked about a minute before the scheduled start time the invite email suggested we use a digital background for our own privacy most felt comfortable

In [7]:
count0 = 0
count1 = 0
for i in y:
    if i == 1:
        count1+=1
    else:
        count0+=1
print(count0)
print(count0/(count1+count0))

19810
0.6429730606945797


In [8]:
#Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=42)

In [9]:
embed = api.load('word2vec-google-news-300')

In [10]:
#Define pretrained model
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dinakartalluri/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#Create the lexicon feature

def lexicon_feature(embed, tokens, word1, word2, t_p, t_n, pos_words, neg_words):
    pos_pole = np.zeros((300,), dtype=float)
    p_count = 0
    neg_pole = np.zeros((300,), dtype=float)
    n_count = 0
    #Found handful of words on thesauras.com
    
    for word in pos_words:
        if word in embed:
            pos_pole+=embed[word]
            p_count+=1
    
    for word in neg_words:
        if word in embed:
            neg_pole+=embed[word]
            n_count+=1
    
    pos_pole/= p_count
    neg_pole/= n_count
            
    length = len(tokens)
    pos_words_count = 0
    neg_words_count = 0
    
    #axis = pos_pole - neg_pole
    axis = embed[word1] - embed[word2]
    
    for token in tokens:
        if token in embed:
            embed_word = embed[token]
            cos_similarity = np.dot(embed_word, axis)/(np.linalg.norm(embed_word)*np.linalg.norm(axis))
            if cos_similarity > t_p:
                pos_words_count+=1
            elif cos_similarity < t_n:
                neg_words_count+=1
    pos_words_count/=length
    neg_words_count/=length
    
    return pos_words_count, neg_words_count

In [ ]:
#feature extraction

#Create features for each review
#First interation
#Interview experience (numeric), difficulty (numeric), avg word embedding for review? (use glove embedding), # of words added to each axis(pick valid threshold)(divide by length of text) (future), use pretrained classifiers (next step)
def extract_features(embed, data, include, sid):
    X_features = np.zeros((len(data), 4))
    if not include:
        X_features = np.zeros((len(data), 3))
    for i, d in enumerate(data):
        feature_1 = 0
        feature_2 = 0

        if d["experience"] == "Positive Experience":
            feature_1 = 1
        elif d["experience"] == "Negative Experience":
            feature_1 = -1

        if d["difficulty"] == "Easy Interview":
            feature_2 = 1
        elif d["difficulty"] == "Difficult Interview":
            feature_2 = -1
        
        tokenized = word_tokenize(d["review"])
        stopwords_english = stopwords.words('english')
        
        feature_3 = 0
        number_embed = 0
        tokens = []
        for token in tokenized:
            if token not in stopwords_english and token not in string.punctuation and token[0].isdigit() == False:
                if token in embed:
                    tokens.append(token)
                    feature_3+=embed[token]
                    number_embed+=1
        feature_3/=number_embed
        feature_3 = np.array(feature_3).mean()
        
        out = sid.polarity_scores(d["review"])
        feature_4 = out['compound']
        
        pos_words = ["accomplishment", "advance", "benefit", "gain", "happiness", "progress", "triumph", "victory", "win"]
        neg_words = ["breakdown", "collapse", "decline", "defeat", "loss", "misstep", "deterioration"]
        pos_count, neg_count = lexicon_feature(embed, tokens, "success", "failure", 0.2, -0.2, pos_words, neg_words)
        #Check the ranges of +-0.3 and +-0.2 to figure out which ones look better (+-0.2 much better)
        #Use feature selection to check if we should use synonyms or not, which one will help performance?
        print(pos_count, neg_count)
        
        
        X_features[i] = np.array([feature_1, feature_2, feature_3, feature_4])
    return X_features
            

In [45]:
#Get the features

X_train_features_with_i = extract_features(embed, X_train, True, sid)
X_test_features_with_i = extract_features(embed, X_test, True, sid)
X_train_features_with_i[100]


0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.009259259259259259
0.016 0.0
0.0 0.0
0.0 0.0
0.0 0.05
0.0 0.0
0.0 0.0
0.0 0.07894736842105263
0.0 0.016666666666666666
0.0 0.010416666666666666
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.034482758620689655
0.0 0.0
0.0 0.038461538461538464
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.058823529411764705
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.06666666666666667 0.0
0.0 0.05
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.03571428571428571 0.0
0.0 0.0625
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.007352941176470588 0.0
0.0 0.0
0.015873015873015872 0.0
0.0 0.0
0.0 0.016666666666666666
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0064516129032258064
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03571428571428571
0.0 0.0
0.0 0.0
0.0 0.044444444444444446
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05263157894736842
0.0 0.05555555555555555
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.014084507042253521 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.047619047619047616 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.07142857142857142
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.04
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.034482758620689655
0.0 0.008849557522123894
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.1
0.0 0.02702702702702703
0.006060606060606061 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.04
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.04
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.006134969325153374 0.006134969325153374
0.0 0.0
0.0 0.03076923076923077
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.038461538461538464
0.0 0.0
0.0 0.0
0.0 0.034482758620689655
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.047619047619047616 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.04
0.0 0.0
0.0 0.0
0.0 0.0
0.014285714285714285 0.0
0.0 0.0
0.0 0.0
0.0625 0.0
0.0333333333333

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0070921985815602835
0.0136986301369863 0.0
0.0 0.0
0.0 0.05555555555555555
0.0 0.0
0.0 0.0
0.0 0.0
0.011235955056179775 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.09090909090909091
0.0 0.0
0.0 0.0
0.0 0.0
0.016129032258064516 0.0
0.0 0.047619047619047616
0.0 0.058823529411764705
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.037037037037037035
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0072992700729927005 0.0024330900243309003
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0025188916876574307
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.06451612903225806
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05555555555555555
0.0 0.0189873417721519
0.0 0.0
0.0 0.0
0.0 0.03636363636363636
0.018518518518518517 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 

0.0 0.0
0.0 0.06060606060606061
0.0 0.0
0.0196078431372549 0.00980392156862745
0.03333333333333333 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0049261083743842365
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.04
0.0 0.02702702702702703
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0041841004184100415 0.0041841004184100415
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.006134969325153374 0.012269938650306749
0.0 0.047619047619047616
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.07692307692307693 0.0
0.0 0.0
0.0 0.0
0.006134969325153374 0.0
0.0 0.0
0.0 0.0
0.011627906976744186 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.022727272727272728
0.0 0.014285714285714285
0.0 0.01098901098901099
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.06666666666666667
0.0 0.0
0.0

0.003968253968253968 0.003968253968253968
0.0 0.0
0.0 0.1
0.0 0.05
0.0 0.04
0.0 0.0
0.0 0.02564102564102564
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.010416666666666666
0.0 0.0625
0.0 0.0
0.0 0.017857142857142856
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.009708737864077669
0.047619047619047616 0.047619047619047616
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0625
0.0 0.05263157894736842
0.0 0.0
0.0 0.03333333333333333
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.01694915254237288 0.0
0.011904761904761904 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03225806451612903
0.0 0.0
0.017543859649122806 0.0
0.0 0.0
0.018518518518518517 0.0
0.007722007722007722 0.003861003861003861
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.047619047619047616
0.0 0.022222222222222223
0.0 0.02702702702702703
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.010416666666666666 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0

...

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.009708737864077669
0.0 0.05
0.0 0.029411764705882353
0.0 0.0
0.0 0.027777777777777776
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.009259259259259259 0.009259259259259259
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.011049723756906077
0.01098901098901099 0.0
0.010526315789473684 0.0
0.0 0.0
0.0 0.07142857142857142
0.0 0.0
0.0 0.0
0.0 0.016129032258064516
0.0 0.15
0.0 0.0
0.0 0.0
0.0 0.03571428571428571
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.09523809523809523 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.012987012987012988
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.037037037037037035
0.0 0.0
0.0 0.05555555555555555
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03333333333333333
0.0 0.0
0.0185185185

0.011904761904761904 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.005434782608695652 0.0
0.03333333333333333 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.03125 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05555555555555555
0.0 0.0055248618784530384
0.0 0.0
0.0 0.05555555555555555
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.045454545454545456
0.0 0.06666666666666667
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.14285714285714285
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0196078431372549
0.0 0.0
0.03125 0.020833333333333332
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.013888888888888888
0.0 0.0
0.0 0.1
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03333333333333333
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.047619047619047616
0.0 0.0
0.004651162790697674 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.018518518518518517
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05
0.0 0.0
0.0 0.0
0.0 0.06060606060606061
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.

0.012345679012345678 0.0
0.0 0.03333333333333333
0.0 0.06451612903225806
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.02857142857142857
0.0 0.0
0.0 0.023255813953488372
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.011111111111111112
0.07142857142857142 0.0
0.0 0.0
0.0 0.0
0.020833333333333332 0.0
0.0 0.0
0.0 0.0
0.0 0.047619047619047616
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.05
0.0 0.05555555555555555
0.0 0.023255813953488372
0.0 0.0
0.0 0.0
0.008849557522123894 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03125
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0196078431372549 0.0
0.045454545454545456 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.004405286343612335 0.004405286343612335
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.03571428571428571
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.034482758620689655
0.0 0.0
0.0 0.0
0.0 0.03448275862

0.0 0.0
0.0 0.0
0.0 0.0


array([ 1.        , -1.        , -0.00293171,  0.9979    ])

In [21]:
X_train_features_with_i[77]

array([ 1.        ,  0.        , -0.00661207, -0.4019    ])

In [113]:
#fit LR, tuning
params={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
lr=LogisticRegression(solver='liblinear')
lr_cv=GridSearchCV(lr,params,cv=10)
lr_cv.fit(X_train_features_with_i,y_train)
print("accuracy :",lr_cv.best_score_)
print("best parameters ",lr_cv.best_params_)

accuracy : 0.6705748259098674
best parameters  {'C': 0.01, 'penalty': 'l2'}


In [119]:
lr=LogisticRegression(C=0.01,penalty="l2")
lr.fit(X_train_features_with_i,y_train)
y_pred = lr.predict(X_test_features_with_i)
print(f1_score(y_test, y_pred, average='weighted'))
print(f1_score(y_test, y_pred, average='micro'))
print(accuracy_score(y_test, y_pred))

0.6205408684421984
0.6598151062155783
0.6598151062155783
